# Train GTAV Self-Driving Model

In [1]:
# Import relevant packages
import random
import time

import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from data.manipulate_image import zoom, img_random_flip, pan, img_random_brightness, random_augment
from data.augmented_dataset import Augmented_Dataset
from trained_models.pilot_net import PilotNet

# Establishing devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")


Using cuda


In [2]:
# Establish fixed values
WIDTH = 160
HEIGHT = 120


In [ ]:
# Load Data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

train_data = np.concatenate((np.load(
    f'drive/My Drive/training_data_angle/training_data-{i}.npy', allow_pickle=True) for i in range(1, 26)))

print("Train Data: ", train_data.shape)


In [ ]:
# Separate data into steering and throttle
steering = []
throttle = []
for data in train_data:
    image = data[0]
    steering_data = data[1][0]
    throttle_data = data[1][1]

    choice = [steering_data, throttle]
    steering.append(steering_data)
    throttle.append(throttle_data)

steering = np.array(steering)
throttle = np.array(throttle)


In [ ]:
# Plot Unbalanced Steering Data
num_bins = 25
samples_per_bin = 3500
hist, bins = np.histogram(steering, num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)
plt.plot((np.min(steering), np.max(steering)),
         (samples_per_bin, samples_per_bin))


In [ ]:
# Plot Unbalanced throttle Data
num_bins = 25
samples_per_bin = 3500
hist, bins = np.histogram(throttle, num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)


In [ ]:
# Balance the steering data
print('total data', len(train_data))
remove_list = []
for j in range(num_bins):
    list_ = []
    for i in range(len(steering)):
        if steering[i] >= bins[j] and steering[i] <= bins[j+1]:
            # print(i)
            list_.append(i)
    list_ = shuffle(list_)
    list_ = list_[samples_per_bin:]
    remove_list.extend(list_)

print('removed', len(remove_list))

print(np.max(remove_list))

train_data = np.delete(train_data, remove_list, axis=0)

print('remaining:', len(train_data))


In [ ]:
# Separate balanced data into steering and throttle
steering = []
throttle = []

for data in train_data:
    image = data[0]
    steering_data = data[1][0]
    throttle_data = data[1][1]
    steering.append(steering_data)
    throttle.append(throttle_data)

steering = np.array(steering)
throttle = np.array(throttle)


In [ ]:
# Plot balanced Steering Data
num_bins = 25
samples_per_bin = 2000
hist, bins = np.histogram(steering, num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)


In [ ]:
# Plot balanced throttle Data
num_bins = 25
samples_per_bin = 2000
hist, bins = np.histogram(throttle, num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)


In [ ]:
# Add augmented data to the dataset
temp_train_data = []
for data in train_data:
    image = data[0]
    choice = data[1]
    flipped_image, flipped_choice = img_random_flip(image, choice)
    temp_train_data.append([flipped_image, flipped_choice])

temp_train_data = np.array(temp_train_data)
train_data = np.concatenate((train_data, temp_train_data))


print("New Size of Training Data:", len(train_data))
train_data = shuffle(train_data)


In [ ]:
# Separate artificially expanded data into steering and throttle
train_data = shuffle(train_data)
steering = []
throttle = []

for data in train_data:
    image = data[0]
    steering_data = data[1][0]
    throttle_data = data[1][1]
    steering.append(steering_data)
    throttle.append(throttle_data)

steering = np.array(steering)
throttle = np.array(throttle)


In [ ]:
# Plot expanded-balanced Steering Data
num_bins = 25
samples_per_bin = 2000
hist, bins = np.histogram(steering, num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)


In [ ]:
# Plot Throttle
hist, bins = np.histogram(throttle, num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)


In [ ]:
# Split the dataset
train = train_data[:-round(len(train_data)*0.20)]
test = train_data[-round(len(train_data)*0.20):]
print("train: ", train.shape)
print("test: ", test.shape)


X = np.array([i[0] for i in train]).reshape(-1, WIDTH, HEIGHT, 1)
Y = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test]).reshape(-1, WIDTH, HEIGHT, 1)
test_y = np.array([i[1] for i in test])


In [ ]:
# Image augmentation used to generalize the performance of the model
# Below is an example of the zoom augmentation
image, choice = train[random.randint(0, len(train)-1)]
zoomed_image = zoom(image)

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axs[0].imshow(image)
axs[0].set_title('Original Image')
axs[1].imshow(zoomed_image)
axs[1].set_title('Zoomed Image')


In [ ]:
# Example of panned image
image, choice = train[random.randint(0, len(train)-1)]
panned_image = pan(image)

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axs[0].imshow(image)
axs[0].set_title('Original Image')
axs[1].imshow(panned_image)
axs[1].set_title('Panned Image')


In [ ]:
# Example of random brightness agumentation
image, choice = train_data[random.randint(0, len(train_data)-1)]
bright_image = img_random_brightness(image)

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axs[0].imshow(image)
axs[0].set_title('Original Image')

axs[1].imshow(bright_image)
axs[1].set_title('bright_image')


In [ ]:
# Example of random flip augmentation
image, choice = train_data[random.randint(0, len(train_data)-1)]
flip_image, flip_choice = img_random_flip(image, choice)

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axs[0].imshow(image)
axs[0].set_title('Original Image')

axs[1].imshow(flip_image)
axs[1].set_title('flip_image')

print("actual steering angle = ", choice[0])
print("flipped steering angle = ", flip_choice[0])


In [ ]:
image, choice = train_data [random.randint(0,len(train_data)-1)]
random_augmented_image, augmented_choice = random_augment(image, choice)

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axs[0].imshow(image) #cmap='gray'
axs[0].set_title('Original Image')

axs[1].imshow(random_augmented_image) #cmap='gray'
axs[1].set_title('Random Augmented Image')

In [ ]:
# Create Datasets and Dataloaders
train_dataset = Augmented_Dataset(train, True, device, WIDTH, HEIGHT)
test_dataset = Augmented_Dataset(test, False, device, WIDTH, HEIGHT)

train_dataloader = DataLoader(train_dataset, batch_size = 300, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 100, shuffle = False)

In [ ]:
# Train and Test Functions
def train(model, train_dataloader, optimizer, criterion, learning_rate, epoch):
    model.train()
    total_loss = 0.
    start_time = time.time()

    for batch_num, (images, choices) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(images)

        loss = criterion(output, choices)
        loss.backward()

        optimizer.step()

        total_loss += loss.item()
        log_interval = 10
        if batch_num % log_interval == 0 and batch_num > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f}'.format(
                      epoch, batch_num, len(
                          train_data) // train_dataloader.batch_size,
                      learning_rate, elapsed * 1000 / log_interval,
                      cur_loss))
            total_loss = 0
            start_time = time.time()


def evaluate(eval_model, test_dataloader):
    eval_model.eval()
    total_num_correct = 0

    with torch.no_grad():
        for (images, choices) in test_dataloader:
            output = eval_model(images)
            total_num_correct += num_correct(output, choices)

    return total_num_correct


def num_correct(output, choices):
    print(output.shape)
    print(choices.shape)


In [ ]:
# Train model
epochs = 25
learning_rate = 0.001

pilot_net = PilotNet(2).to(device)
criterion = nn.MSELoss()
optimizer = Adam(pilot_net.parameters, learning_rate)

best_val_loss = float("inf")
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(pilot_net, train_dataloader, optimizer,
          criterion, learning_rate, epoch)
    val_loss = evaluate(pilot_net, test_dataloader)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = pilot_net


In [ ]:
# Save model
torch.save(best_model.state_dict(), "trained_models\pilot_model.pt")